In [1]:
import os
import pandas as pd
from openai import OpenAI
import textwrap
# client = OpenAI(api_key = api_key)

In [2]:
type(client)

openai.OpenAI

In [146]:
system_content_cot = '''Assume you are a chess master.
    You will be provided with a list of chess move pairs in Algebraic Notation where 1st move is by White and 2nd by Black.
    Your task is to analyse each pair and then generate the rationale behind each White and Black move and then the last move.
    '''

In [85]:
df = pd.read_csv("./data/progressive_moves_500.csv", index_col=0)
df.drop("opening_name", axis = 1, inplace=True)
df = df.fillna("None")
df["explanation"] = " "
df.head()

,prev_moves,last_move,explanation
0,None,d4,
1,d4,d5,
2,d4 d5,c4,
3,d4 d5 c4,c6,
4,d4 d5 c4 c6,cxd5,


In [86]:
df.shape[0]

500

In [162]:
def process_response(response):
    lines = response.replace("", "").replace("", "").split("\n")
    wrapped_lines = [textwrap.fill(line, width=100) for line in lines]
    for wrapped_line in wrapped_lines:
        print(wrapped_line)

In [87]:
def gen_cot_prompt(prev_moves : str, last_move : str) -> str:
    if prev_moves == "None":
        return f"White : {last_move}"
    else: 
        prev_moves = (prev_moves+f" {last_move}").split(" ")
        cot_moves = [f"White: {prev_moves[i]}, Black: {prev_moves[i+1]}" if i + 1 < len(prev_moves) else f"White: {prev_moves[i]}" for i in range(0, len(prev_moves), 2)]
        return "\n".join(cot_moves)

In [156]:
temp = df.head().copy()
temp

,prev_moves,last_move,explanation
0,None,d4,
1,d4,d5,
2,d4 d5,c4,
3,d4 d5 c4,c6,
4,d4 d5 c4 c6,cxd5,


In [157]:
def generate_explanation(df : pd.DataFrame, sytem_content : str) -> pd.DataFrame:
    return_df = df.copy()

    for i in range(df.shape[0]):
        prev_moves = df.iloc[i]["prev_moves"]
        last_move = df.iloc[i]["last_move"]

        user_prompt_move = gen_cot_prompt(prev_moves, last_move)
        user_content_cot = f"Previous move pairs are - \n{user_prompt_move}"
        
        stream = client.chat.completions.create(
        model="gpt-3.5-turbo-0125",
        messages = [{"role": "system",
                     "content": sytem_content},
                    {"role": "user",
                     "content": user_content_cot}],
        stream=True,)
        
        explanation = ""
        for chunk in stream:
            if chunk.choices[0].delta.content is not None:
                explanation += chunk.choices[0].delta.content
        return_df.loc[i, "explanation"] = explanation
    return return_df


In [158]:
temp = generate_explanation(temp, system_content_cot)
temp

,prev_moves,last_move,explanation
0,None,d4,"Black: e5\n\nAnalysis:\n1. White played d4, a ..."
1,d4,d5,White's move of playing d4 is a common opening...
2,d4 d5,c4,White move: c4\n\nRationale behind White's mov...
3,d4 d5 c4,c6,"In the first move pair, White played d4, which..."
4,d4 d5 c4 c6,cxd5,"In the first move pair, White played d4, aimin..."


In [163]:
process_response(temp.loc[4, "explanation"])

In the first move pair, White played d4, aiming to control the center of the board and open up lines
for the queen and bishop. Black responded with d5, mirroring White's move by also aiming to control
the center and contest White's influence.

In the second move pair, White played c4, further reinforcing their control over the center and
preparing to develop the bishop on the c1-h6 diagonal. Black played c6, preparing to support the
pawn on d5 and potentially fianchetto the bishop on b7.

In the third move, White played cxd5, capturing Black's d5 pawn with the c4 pawn. This move helps
White gain more central control by exchanging pawns and potentially opening lines for their pieces.
Black now has the option to recapture the pawn on d5 or make a different move based on the current
board position.
